In [3]:
import pandas as pd
import sweetviz as sv
import numpy as np
import re
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df_user = pd.read_csv('./csv/dtt_users.csv')
df_country = pd.read_csv('./csv/dtt_country.csv')[['country_id','country_code','country_phonecode','country_name']]

order_col = [
    'customer_id','customer_firstname','customer_lastname','customer_email','customer_gender',
    'customer_nationality','customer_country','customer_phone_iso','customer_phone_code',
    'browser', 'platform',
    'order_product_id','order_price_paid','order_state','order_payment_by'
    ]
df_order = pd.read_csv('./csv/dtt_order.csv')[order_col]

countryCode_to_id = {i[1][1] : i[1][0] for i in df_country.iterrows()}
phoneCode_to_id = {}
countryName_to_id = {i[1][3] : i[1][0] for i in df_country.iterrows()}
for i in df_country.iterrows():
    val = i[1][0]
    if len(i[1][2].split(',')) > 1:
        for j in i[1][2].split(','):
            phoneCode_to_id[j] = val
    else:
        phoneCode_to_id[i[1][2]] = val

/var/folders/5g/90_bh_2146gffg2__wzbyb780000gn/T/ipykernel_87275/2848133556.py:10: DtypeWarning: Columns (2,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_order = pd.read_csv('./csv/dtt_order.csv')[order_col]


In [5]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def encode_gender(inputString):
    if 'Mr.' == inputString or 'mr.' in inputString:
        return 0
    elif 'Ms.' == inputString or 'ms.' in inputString:
        return 1
    elif 'Mrs.' == inputString or 'mrs.' in inputString:
        return 2
    else:
        return 3

def encode_phone_code(inputFloat):
    try:
        return phoneCode_to_id[str(int(inputFloat))]
    except:
        print(int(inputFloat))

def encode_order_payment_by(inputString):
    if 'creditcard' == inputString:
        return 1
    elif 'paypal' == inputString:
        return 2
    else:
        return 0

In [35]:
df_order_v1 = df_order[order_col]
# drop na
na_index_v1 = df_order_v1[df_order_v1['customer_country'].isna()].index
df_order_v1 = df_order_v1.drop(na_index_v1)

# drop test
test_index_v1 = df_order_v1[df_order_v1['customer_email'].map(lambda x: 'test' in str(x) or '@mail.com' in str(x) or '360' in str(x))].index
df_order_v1 = df_order_v1.drop(test_index_v1)

test_index_v2 = df_order_v1[df_order_v1['customer_email'].map(lambda x: 'mitkung' in str(x) or 'suphattra' in str(x))].index
df_order_v1 = df_order_v1.drop(test_index_v2)

test_index_v3 = df_order_v1[df_order_v1['customer_firstname'].map(lambda x: 'admin' in str(x) or 'test' in str(x) or 'Thanadol' in str(x) or 'Test' in str(x))].index
df_order_v1 = df_order_v1.drop(test_index_v3)

test_index_v3 = df_order_v1[df_order_v1['customer_firstname'].map(lambda x: ('asd' in str(x) or 'dsd' in str(x) or 'xx' in str(x)) and not 'Jasdeep' in str(x))].index
df_order_v1 = df_order_v1.drop(test_index_v3)

# clean
df_order_v1.loc[3627, 'customer_firstname'] = 'Jennyhan'
df_order_v1['customer_firstname'] = df_order_v1['customer_firstname'].map(lambda x: re.sub('[0-9#$;:]+', '', x))
df_order_v1['customer_lastname'] = df_order_v1['customer_lastname'].map(lambda x: re.sub('[0-9#$;:]+', '', x))
df_order_v1['customer_nationality'] = df_order_v1['customer_nationality'].map(lambda x: re.sub('ไทย|Thailand', 'thai', x))
df_order_v1['customer_nationality'] = df_order_v1['customer_nationality'].map(lambda x : x.lower())

# encode
df_order_v1['customer_gender'] = df_order_v1['customer_gender'].map(lambda x : encode_gender(str(x)))
df_order_v1['customer_country'] = df_order_v1['customer_country'].map(lambda x : countryName_to_id[x] if x != '' else x)

df_order_v1.fillna(value={'customer_phone_iso': -1, 'customer_phone_code':-1, 'order_payment_by': 0}, inplace=True)
df_order_v1['customer_phone_iso'] = df_order_v1['customer_phone_iso'].map(lambda x : -1 if x == -1 else countryCode_to_id[x])
df_order_v1['customer_phone_code'] = df_order_v1['customer_phone_code'].map(lambda x : -1 if x == -1 else encode_phone_code(x))
df_order_v1['order_payment_by'] = df_order_v1['order_payment_by'].map(lambda x : encode_order_payment_by(str(x)))

df_order_v1['customer_phone_iso'] = np.where(df_order_v1['customer_phone_iso']==-1, df_order_v1['customer_country'], df_order_v1['customer_phone_iso'])
df_order_v1['customer_phone_code'] = np.where(df_order_v1['customer_phone_code']==-1, df_order_v1['customer_country'], df_order_v1['customer_phone_iso'])

enc = OrdinalEncoder(dtype='int8', encoded_missing_value=-1)
cat_cols = ['browser', 'platform', 'order_state']
df_order_v1[cat_cols] = enc.fit_transform(df_order_v1[cat_cols])

# change dtype to int
int_cols = ['customer_phone_iso', 'customer_phone_code','order_price_paid', 'customer_id']
df_order_v1['customer_phone_iso'] = np.where(df_order_v1['customer_phone_iso']==-1, df_order_v1['customer_country'], df_order_v1['customer_phone_iso'])
df_order_v1['customer_phone_code'] = np.where(df_order_v1['customer_phone_code']==-1, df_order_v1['customer_country'], df_order_v1['customer_phone_iso'])
df_order_v1[int_cols] = df_order_v1[int_cols].astype('int64')

# df_order_v1['order_price_paid'] = df_order_v1['order_price_paid'].astype('int64')
# df_order_v1['customer_id'] = df_order_v1['customer_id'].astype('int64')

# test_index_v4 = df_order_v1[df_order_v1['customer_email'].map(lambda x: 'daytriptour' in str(x) or 'support' in str(x))].index
# df_order_v1 = df_order_v1.drop(test_index_v4)

# test_index_v5 = df_order_v1[df_order_v1['customer_email'].map(lambda x: 'dolavic' in str(x) or 'thanadol' in str(x))].index
# df_order_v1 = df_order_v1.drop(test_index_v5)

In [36]:
ls_encode = []
for i in enc.categories_:
    ls_encode.append(i.tolist())
ls_encode

[['Android Browser',
  'AppleWebKit',
  'Chrome',
  'Edge',
  'Firefox',
  'MSIE',
  'Opera Next',
  'Safari',
  'SamsungBrowser',
  'Vivaldi',
  nan],
 ['Android',
  'Chrome OS',
  'Linux',
  'Macintosh',
  'Windows',
  'iPad',
  'iPhone',
  nan],
 ['cancelled', 'detail_success', 'payment_success']]

In [42]:
# report = sv.analyze(df_order_v1)
# report.show_html('./report/clean_order_v1.html')
# df_order_v1.to_csv('./csv/clean_order_v1.csv')

In [38]:
df_order_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9250 entries, 1285 to 27326
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   customer_id           9250 non-null   int64 
 1   customer_firstname    9250 non-null   object
 2   customer_lastname     9250 non-null   object
 3   customer_email        9250 non-null   object
 4   customer_gender       9250 non-null   int64 
 5   customer_nationality  9250 non-null   object
 6   customer_country      9250 non-null   int64 
 7   customer_phone_iso    9250 non-null   int64 
 8   customer_phone_code   9250 non-null   int64 
 9   browser               9250 non-null   int8  
 10  platform              9250 non-null   int8  
 11  order_product_id      9250 non-null   int64 
 12  order_price_paid      9250 non-null   int64 
 13  order_state           9250 non-null   int8  
 14  order_payment_by      9250 non-null   int64 
dtypes: int64(8), int8(3), object(4)
me

In [40]:
df_order_v1

,customer_id,customer_firstname,customer_lastname,customer_email,customer_gender,customer_nationality,customer_country,customer_phone_iso,customer_phone_code,browser,platform,order_product_id,order_price_paid,order_state,order_payment_by
1285,0,Julie,Staplea,juliestaples40@gmail.com,2,british,77,77,77,2,0,121,0,1,0
1286,0,Julie,Staples,juliestaples40@gmail.com,2,british,77,77,77,2,0,121,4500,2,0
1296,0,Chad,Steedman,chad.steedman@hotmail.com,0,australian,13,13,13,7,6,55,3440,2,0
1300,0,Dale,Lewis,juleslewis@hotmail.co.uk,0,welsh,77,77,77,7,6,90,0,1,0
1303,0,Samer,Zainal,s.zainal@gmail.com,0,bahrain,23,23,23,2,0,127,699,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27311,2824,Ann Mary,Baby,annmarybaby5@gmail.com,0,india,105,105,105,2,4,122,0,1,0
27318,0,Reneta,Gesheva,rgesheva8@gmail.com,2,american,233,233,233,7,6,202,4280,2,0
27320,0,Odmunkh,Ebkhbold,odmunkh@yahoo.com,0,mongolia,147,147,147,2,0,195,0,1,0
27323,2826,Harsh Vardhan,Rathi,harshrathi1119@gmail.com,0,indian,105,105,105,7,6,123,0,1,0
